# 2장
### url 추출

In [1]:
# 지정한 URL 웹 페이지 추출
from urllib.request import urlopen
f = urlopen('http://www.naver.com')

In [2]:
# HTTPResponse 자료형의 객체
type(f)

http.client.HTTPResponse

In [3]:
# 본문 추출
f.read()[:1000]
#read(): Reads and returns the response body, or up to the next amt bytes.
#한번 읽으면 다시 return 못하는 듯.

b'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="\xeb\x84\xa4\xec\x9d\xb4\xeb\xb2\x84 \xeb\xa9\x94\xec\x9d\xb8\xec\x97\x90\xec\x84\x9c \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xec\xa0\x95\xeb\xb3\xb4\xec\x99\x80 \xec\x9c\xa0\xec\x9a\xa9\xed\x95\x9c \xec\xbb\xa8\xed\x85\x90\xec\xb8\xa0\xeb\xa5\xbc \xeb\xa7\x8c\xeb\x82\x98 \xeb\xb3\xb4\xec\x84\xb8\xec\x9a\x94"/>\n<meta property="og:title" content="\xeb\x84\xa4\xec\x9d\xb4\xeb\xb2\x84">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s

In [4]:
# 상태 코드 추출
f.status

200

### 인코딩 방식을 추출하고 그에 맞게 디코딩하기

In [5]:
# HTTP 헤더의 값 추출 -> Content-Type헤더를 참조하면 인코딩 방식을 알 수 있고, 이에 따라 문자코드(UTF-8, EUC-KR)를 지정해서 디코딩
f.getheader('Content-Type')

'text/html; charset=UTF-8'

In [6]:
# encoding 방식(utf-8) 추출
encoding = f.info().get_content_charset(failobj='UTF-8')
encoding

'utf-8'

In [7]:
# 추출한 encoding방식으로 decoding
f = urlopen('http://www.naver.com')
text = f.read().decode(encoding)
text[:1000]

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

- Content-Type 헤더에서 인코딩 정보가 항상 맞는 것은 아니므로 
- 위의 방법으로 UnicodeDecodeError가 발생하면 
- meta 태그 또는 응답 본문의 바이트열도 확인하여 인코딩 방식을 결정해야한다.
- `<meta charset="utf-8">` 이런 식으로 앞부분에 적혀 있음.

In [8]:
import re
import sys

In [9]:
f = urlopen("http://www.naver.com")
bytes_content = f.read()

# 앞부분에 대체로 meta태그가 있기 때문에 앞부분만 디코딩하고, 정규표현식으로 charset값을 추출
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')
scanned_text

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="��������� ������������ ��������� ��������� ��������� ������������ ������ ���������"/>\n<meta property="og:title" content="���������">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="��������� ������������ ��������� ��������� ��������� ������������ ������ ���������"/>\n<meta name="twitter:card" content="summary">\n<meta 

In [10]:
# re.search는 밑에서 자세히
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
print(match)
print(match.group(1))

<_sre.SRE_Match object; span=(62, 76), match='charset="utf-8'>
utf-8


In [11]:
bytes_content.decode(match.group(1))[:1000]

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

### 스크레이핑 - 정규표현식 / XML파서 이용
- (궁금해서 찾아본) HTML과 XML 비교 [참고](https://www.crocus.co.kr/1493)
    + 둘 다 마크업 언어(태그 등을 이용하여 문서나 데이터의 구조를 명기하는 언어)
    + 목적이 다른데, 
        * XML은 데이터 저장과 전송을 목적으로,
        * HTML은 데이터를 웹상에 표현하기 위한 목적.
    + 태그 차이도 존재
        * HTML은 태그가 정해져있으나 XML은 미리 정의된 태그가 없음.
        * HTML은 태그 사양이 더 유연함. 생략을 한다던지. 웹 브라우저는 문법에 문제가 있는 HTML이라도 출력해줌.
    + 추가적으로 
        * HTML은 웹 환경에서 작동되는 언어, XML은 환경에 구애받지 않음.
        * HTML은 데이터+표현 동시에, XML은 데이터만.

1. 정규표현식 [정규표현식 연습 사이트](https://pythex.org)
    - HTML을 문자열으로 취급. 
    - 마크업되지 않은 웹 페이지도 스크레이핑 가능

In [12]:
# re모듈은 정규표현식 패턴의 \ 처리를 도와줌.
re.search(r'a.*c', 'abc123DEF')

# 두번째 매개변수에 첫번째 정규표현식 부분을, Match객체로 반환. 안맞으면 None return.

<_sre.SRE_Match object; span=(0, 3), match='abc'>

In [13]:
re.search(r'a.*d', 'abc123DEF', re.IGNORECASE)

# re.IGNORECASE : 대소문자 구분 무시

<_sre.SRE_Match object; span=(0, 7), match='abc123D'>

In [14]:
m = re.search(r'a(.*)c', 'abc123DEF')
print(m.group(0)) # 0이면 모든 값 반환.
print(m.group(1)) # 1이면 ()안에 있는 값 반환.

abc
b


In [15]:
re.findall(r'\w{2,}', 'This is a pen') #2글자 이상의 단어 추출

['This', 'is', 'pen']

In [16]:
re.sub(r'\w{2,}', 'That', 'This is a pen') # 대체

'That That a That'

In [17]:
f = urlopen("http://www.hanbit.co.kr/store/books/full_book_list.html")
html = f.read().decode('utf-8')

for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL)[:6] :
    print('html :', partial_html)
    
    url = re.search(r'<a href="(.*?)">', partial_html).group(1)
    url = 'http://www.hanbit.co.kr' + url
    print('url :',url)
    
    title = re.sub(r'<.*?>', '', partial_html)
    title = re.sub(r'&#40;', '(', title)
    title = re.sub(r'&#41;', ')', title)
    print('title :',title)
    print("-----------")

html : <td class="left"><a href="/store/books/look.php?p_code=B6498472179">죽음의 부정</a></td>
url : http://www.hanbit.co.kr/store/books/look.php?p_code=B6498472179
title : 죽음의 부정
-----------
html : <td class="left"><a href="/store/books/look.php?p_code=B8548562053">IT CookBook , MySQL로 배우는 데이터베이스 개론과 실습</a></td>
url : http://www.hanbit.co.kr/store/books/look.php?p_code=B8548562053
title : IT CookBook , MySQL로 배우는 데이터베이스 개론과 실습
-----------
html : <td class="left"><a href="/store/books/look.php?p_code=B7446987985">숨은 그림 찾기&#40;똑똑한 두뇌연습 만 4~6세&#41;</a></td>
url : http://www.hanbit.co.kr/store/books/look.php?p_code=B7446987985
title : 숨은 그림 찾기(똑똑한 두뇌연습 만 4~6세)
-----------
html : <td class="left"><a href="/store/books/look.php?p_code=B5675781518">리얼 홍콩 마카오 [2019~2020년 개정판]</a></td>
url : http://www.hanbit.co.kr/store/books/look.php?p_code=B5675781518
title : 리얼 홍콩 마카오 [2019~2020년 개정판]
-----------
html : <td class="left"><a href="/store/books/look.php?p_code=B6797717479">Head First Agile : 개념부터

2. XML파서
    - XML태그를 분석(파싱)하고 필요한 부분을 추출.
    - 정규표현식보다 간단하게 추출 가능.
    - HTML이 XML보다 유연하기 때문에 문제가 있는 HTML을 파싱하는데 어려움이 있을 수 있음.
    - HTML 파서는 없나 생각했는데, python 표준 모듈 html.parser모듈이 있다고. 그러나 번거롭다고 한다.

- RSS란? 블로그나 뉴스 같은 웹사이트는 변경 정보등을 RSS라는 이름으로 제공. 이는 XML형식으로 제공됨.
- 따라서, 스크레이핑하고 싶은 정보가 RSS로 제공되면 XML파서로 스크레이핑 가능.

In [19]:
from xml.etree import ElementTree

# parse로 파일을 읽음
tree = ElementTree.parse('rss.xml')
tree

In [20]:
# XML tree의 루트를 찾고
root = tree.getroot()
root

<Element 'rss' at 0x00000283D627ECC8>

In [21]:
# 주어진 파일에서 시간과 날씨를 찾기위해 root에서 해당 태그로 내려감
# root > channel > item > description > body > location > data
for item in root.findall('channel/item/description/body/location/data')[:6] : 
    tm_ef = item.find('tmEf').text # text로 텍스트 값 저장
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    print(tm_ef, tmn, tmx, wf)

2019-08-21 00:00 24 30 구름많음
2019-08-21 12:00 24 30 흐리고 비
2019-08-22 00:00 23 30 구름많고 비
2019-08-22 12:00 23 30 구름많음
2019-08-23 00:00 22 30 구름많음
2019-08-23 12:00 22 30 구름많음
